#### Load necessary libraries

In [ ]:
import os
import librosa
import numpy as np
import pandas as pd
from sklearn.preprocessing import label_binarize
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import Adam
from sklearn import metrics 
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split

#### Filepath for data and some constant

In [260]:
#filepath of file with labels and training set
name = 'C:/Users/Python/Documents/STC_test/data_v_7_stc/meta/meta.txt'
path_training = 'C:/Users/Python/Documents/STC_test/data_v_7_stc/audio/'
#put here filepath for new set
path_test = 'C:/Users/Python/Documents/STC_test/data_v_7_stc/test/' 
n_mfcc = 100
number_fun = 2

#### Reading data

In [261]:
#obtain features and scaling for new set
features_test = np.array([]).reshape(0, n_mfcc*number_fun)
name_test_file = np.array([])
for file in os.listdir(path_test):
    x, sr = librosa.load(os.path.join(path_test,file))
    mfcc = librosa.feature.mfcc(x, sr, n_mfcc = n_mfcc)
    mfcc_mean = np.mean(mfcc.T,axis=0)
    mfcc_median = np.median(mfcc.T,axis=0)
    features_test = np.vstack((features_test, np.concatenate((mfcc_mean, mfcc_median))))
    name_test_file = np.append(name_test_file, file)
test = StandardScaler().fit_transform(features_test)

In [262]:
#transform and obtain features from training audio file 
features = np.array([]).reshape(0, n_mfcc*number_fun)
name_file = np.array([])
for file in os.listdir(path_training):
    x, sr = librosa.load(os.path.join(path_training,file))
    mfcc = librosa.feature.mfcc(x, sr, n_mfcc = n_mfcc)
    mfcc_mean = np.mean(mfcc.T,axis=0)
    mfcc_median = np.median(mfcc.T,axis=0)
    features = np.vstack((features, np.concatenate((mfcc_mean, mfcc_median))))
    name_file = np.append(name_file, file)

In [263]:
#load label of classes
label = pd.read_csv(name, sep = '\t', header=None)
label.columns = ['name', 'place', 'start', 'end', 'class']
features_data = pd.DataFrame(np.concatenate((name_file.reshape(name_file.shape[0],1), features), axis=1))
features_data.columns = np.concatenate((['name'], ['feature_'+str(i) for i in range(1,n_mfcc*number_fun+1)]))
train = pd.merge(label, features_data, on='name')

#### Classifcation

In [237]:
#scaling training set
X = train.drop(['class','name', 'place', 'start', 'end'], axis = 1)
X = StandardScaler().fit_transform(X)

In [238]:
# Binarize the output
y = label_binarize(train['class'], classes=train['class'].unique())

In [249]:
#split set on train and test
train_X, test_X, train_y, test_y = train_test_split(X,y,train_size=0.5, test_size = 0.5,random_state=40)

In [240]:
num_labels = y.shape[1]

In [251]:
# build model
model = Sequential()

model.add(Dense(64, input_shape=(n_mfcc*number_fun,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(num_labels))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [252]:
model.fit(train_X, train_y, batch_size=50, epochs=100, validation_data=(test_X, test_y))
scores = model.evaluate(test_X, test_y)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

Train on 5653 samples, validate on 5654 samples
Epoch 1/100
5653/5653 [==============================] - 1s 237us/step - loss: 2.0866 - acc: 0.2519 - val_loss: 1.4493 - val_acc: 0.5463
Epoch 2/100
5653/5653 [==============================] - 0s 45us/step - loss: 1.6462 - acc: 0.4162 - val_loss: 1.2572 - val_acc: 0.6380
Epoch 3/100
5653/5653 [==============================] - 0s 46us/step - loss: 1.4537 - acc: 0.4897 - val_loss: 1.1130 - val_acc: 0.6970
Epoch 4/100
5653/5653 [==============================] - 0s 45us/step - loss: 1.3192 - acc: 0.5436 - val_loss: 1.0088 - val_acc: 0.7213
Epoch 5/100
5653/5653 [==============================] - 0s 45us/step - loss: 1.2157 - acc: 0.5944 - val_loss: 0.8956 - val_acc: 0.7556
Epoch 6/100
5653/5653 [==============================] - 0s 45us/step - loss: 1.1513 - acc: 0.6096 - val_loss: 0.8275 - val_acc: 0.7720
Epoch 7/100
5653/5653 [==============================] - 0s 47us/step - loss: 1.0887 - acc: 0.6351 - val_loss: 0.7598 - val_acc: 0.7872

5653/5653 [==============================] - 0s 50us/step - loss: 0.4791 - acc: 0.8505 - val_loss: 0.1931 - val_acc: 0.9468
Epoch 61/100
5653/5653 [==============================] - 0s 43us/step - loss: 0.4680 - acc: 0.8516 - val_loss: 0.1977 - val_acc: 0.9438
Epoch 62/100
5653/5653 [==============================] - 0s 44us/step - loss: 0.4708 - acc: 0.8505 - val_loss: 0.1919 - val_acc: 0.9443
Epoch 63/100
5653/5653 [==============================] - 0s 44us/step - loss: 0.4641 - acc: 0.8518 - val_loss: 0.1839 - val_acc: 0.9462
Epoch 64/100
5653/5653 [==============================] - 0s 41us/step - loss: 0.4642 - acc: 0.8530 - val_loss: 0.1822 - val_acc: 0.9430
Epoch 65/100
5653/5653 [==============================] - 0s 45us/step - loss: 0.4460 - acc: 0.8578 - val_loss: 0.1970 - val_acc: 0.9423
Epoch 66/100
5653/5653 [==============================] - 0s 43us/step - loss: 0.4633 - acc: 0.8534 - val_loss: 0.1806 - val_acc: 0.9464
Epoch 67/100
5653/5653 [==============================

#### Prediction and results

In [253]:
test_class = model.predict_classes(test)

values = train['class'].unique()
keys = range(num_labels)
dictionary = dict(zip(keys, values))
test_class = [dictionary.get(i) for i in test_class]

In [254]:
test_prob = model.predict(test)

In [255]:
result = pd.DataFrame()

In [256]:
result['file_name'] = name_test_file
result['predict'] = np.max(test_prob, axis = 1)
result['class'] = test_class

In [257]:
result

,file_name,predict,class
0,background_0001.wav,0.902539,background
1,background_0004.wav,0.949558,background
2,background_0009.wav,0.557214,keyboard
3,background_0012.wav,0.637077,background
4,background_0016.wav,0.915039,background
5,background_0018.wav,0.997444,background
6,background_0022.wav,0.970258,keyboard
7,background_0028.wav,0.994611,keyboard
8,background_0031.wav,0.996014,keyboard
9,background_0036.wav,0.990461,keyboard


In [259]:
result.to_csv('C:/Users/Python/Documents/STC_test/result.txt', sep = '\t', header =False, index = False)